In [1]:
import pandas as pd
import numpy as np
import json

/Users/ilenapeng/.pyenv/versions/3.9.7/lib/python3.9/site-packages/pandas/compat/__init__.py:124: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
df = pd.read_csv('06-geocode.csv')
df.head()

,school,count,circuit,address,lat,lng
0,Harvard University,24,District of Columbia,"Massachusetts Hall, Cambridge, MA 02138",42.37717,-71.13507
1,Yale University,16,District of Columbia,"105 Wall Street, New Haven, CT 06520",41.31115,-72.92677
2,Stanford University,14,District of Columbia,"450 Serra Mall, Stanford, CA 94305-2061",37.42765,-122.17006
3,Duke University,11,District of Columbia,"207 Allen Bldg, Durham, NC 27708",35.99981,-78.94058
4,University of Virginia,9,District of Columbia,"1827 University Avenue, Charlottesville, VA 22904",38.03662,-78.50258


### Create properties for geojson

In [3]:
return_to_home = """<div><a id="myLink" title="Click to return home"href="#" onclick="backHome();return false;">Back to home</a></div>"""

In [4]:
# Create 'article' -- popup text
# Single judge
df.loc[df['count']==1, 'article'] = "<h1>" + df['school'] + "</h1><p>There is " + df['count'].map(str) + " current or former judge in the " + df.circuit + " circuit that has a degree from " + df.school + ".</p>" + return_to_home

# More than one judge
df.loc[df['count']>1, 'article'] = "<h1>" + df['school'] + "</h1>There are " + df['count'].map(str) + " current and former judges in the " + df.circuit + " circuit that have degrees from " + df.school + ".</p>" + return_to_home

In [5]:
# Create 'headline' -- rollover text
df.loc[df['count']==1, 'headline'] = "<p>" + df.circuit + " circuit</p><p>" + df.school + ", 1 judge</p>"

df.loc[df['count']>1, 'headline'] = "<p>" + df.circuit + " circuit</p><p>" + df.school + ", " + df['count'].map(str) + " judges</p>"

In [6]:
# Create name, group_name
# name: rename 'school'
# group_name: rename 'circuit'
df = df.rename(columns={'school': 'name', 'circuit': 'group_name'})

# radius: is count, but with some map
# df['count'] = float(df['count'])
df['radius'] = np.sqrt((df['count']*20)/3.14)

In [7]:
# Create 'group_id'
def create_group_id(row):
    if row['group_name'] == 'First':
      return 1
    if row['group_name'] == 'Second':
      return 2
    if row['group_name'] == 'Third':
      return 3
    if row['group_name']  == 'Fourth':
      return 4
    if row['group_name'] == 'Fifth':
      return 5
    if row['group_name'] == 'Sixth':
      return 6
    if row['group_name'] == 'Seventh':
      return 7
    if row['group_name'] == 'Eighth':
      return 8
    if row['group_name'] == 'Ninth':
      return 9
    if row['group_name'] == 'Tenth':
      return 10
    if row['group_name'] == 'Eleventh':
      return 11
    if row['group_name'] == 'District of Columbia':
      return 12
    if row['group_name'] == 'Federal':
      return 13

df['group_id'] = df.apply (lambda row: create_group_id(row), axis=1)

In [8]:
# color: use 'group_name' ('circuit') to create individual colors
def create_color(row):
    if row['group_name'] == 'First':
      return '#F94144'
    if row['group_name'] == 'Second':
      return '#F3722C'
    if row['group_name'] == 'Third':
      return '#F8961E'
    if row['group_name']  == 'Fourth':
      return '#F9844A'
    if row['group_name'] == 'Fifth':
      return '#F9C74F'
    if row['group_name'] == 'Sixth':
      return '#483A58'
    if row['group_name'] == 'Seventh':
      return '#90BE6D'
    if row['group_name'] == 'Eighth':
      return '#43AA8B'
    if row['group_name'] == 'Ninth':
      return '#4D908E'
    if row['group_name'] == 'Tenth':
      return '#577590'
    if row['group_name'] == 'Eleventh':
      return '#277DA1'
    if row['group_name'] == 'District of Columbia':
      return '#6D597A'
    if row['group_name'] == 'Federal':
      return '#B56576'

df['color'] = df.apply (lambda row: create_color(row), axis=1)

### Create geojson

In [9]:
# From: https://notebook.community/captainsafia/nteract/applications/desktop/example-notebooks/pandas-to-geojson
def df_to_geojson(df, properties, lat='lat', lon='lng'):
    # create a new python dict to contain our geojson data, using geojson format
    geojson = {'type':'FeatureCollection', 'features':[]}

    # loop through each row in the dataframe and convert each row to geojson format
    for _, row in df.iterrows():
        # create a feature template to fill in
        feature = {'type':'Feature',
                   'properties':{},
                   'geometry':{'type':'Point',
                               'coordinates':[]}}

        # fill in the coordinates
        feature['geometry']['coordinates'] = [row[lon],row[lat]]

        # for each column, get the value and add it as a new feature property
        for prop in properties:
            feature['properties'][prop] = row[prop]
        
        # add this feature (aka, converted dataframe row) to the list of features inside our dict
        geojson['features'].append(feature)
    
    return geojson

columns = ['headline', 'count', 'group_name', 'lat', 'lng', 'article', 'name', 'radius', 'group_id', 'color']
geojson = df_to_geojson(df, columns)

In [10]:
with open('07-geojson.js', 'w') as outfile:
    outfile.write("var infoData = ")
#geojson output
with open('07-geojson.js', 'a') as outfile:
    json.dump(geojson, outfile)